# Image Manipulations and Image Spaces

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import color
import os

In [ ]:
# Read an image
original_image = Image.open("./test_images/100007.jpg")

# Show the image
original_image.show()

In [ ]:
# Transform the image to numpy array to manipulate it
image_array = np.array(original_image)

# Inspect the image array
print(f"Shape: {image_array.shape}")
print(f"Red channel:\n{image_array[:, :, 0]}")
print(f"Green channel:\n{image_array[:, :, 1]}")
print(f"Blue channel:\n{image_array[:, :, 2]}")

In [ ]:
# Manipulation example: make the image darker
image_array_v1 = (image_array * 0.6).astype(np.uint8)
image_v1_darker = Image.fromarray(image_array_v1)
image_v1_darker.show()

In [ ]:
# Manipulation example: manipulate the red channel
image_array_v2 = image_array[:, :, 0]
image_v2 = Image.fromarray(image_array_v2)
image_v2.show()
# Why is it grey?


In [ ]:

image_array_v2 = image_array.copy()
image_array_v2[:, :, 1] = 0  # Set green channel to 0
image_array_v2[:, :, 2] = 0  # Set blue channel to 0
image_v2 = Image.fromarray(image_array_v2)
image_v2.show()

In [ ]:
# Transform RGB array to Lab array
from skimage.color import rgb2lab, lab2rgb

image_array_lab = rgb2lab(image_array)
# Inspect the image array
print(f"Shape: {image_array_lab.shape}")
print(f"L channel:\n{image_array_lab[:, :, 0]}")
print(f"a channel:\n{image_array_lab[:, :, 1]}")
print(f"b channel:\n{image_array_lab[:, :, 2]}")

# Transform Lab array to RGB array
image_array_rgb = lab2rgb(image_array_lab)

# Inspect the image array
print(f"Shape: {image_array_rgb.shape}")
print(f"Red channel:\n{image_array_rgb[:, :, 0]}")
print(f"Green channel:\n{image_array_rgb[:, :, 1]}")
print(f"Blue channel:\n{image_array_rgb[:, :, 2]}")

# Show the image
image = Image.fromarray((image_array_rgb * 255).astype(np.uint8))
image.show()

In [ ]:
# Transform RGB to HSV
from skimage.color import rgb2hsv, hsv2rgb

image_array_hsv = rgb2hsv(image_array)
# Inspect the image array
print(f"Shape: {image_array_hsv.shape}")
print(f"H channel:\n{image_array_hsv[:, :, 0]}")
print(f"S channel:\n{image_array_hsv[:, :, 1]}")
print(f"V channel:\n{image_array_hsv[:, :, 2]}")

# Histogram equalization on the V channel
from skimage.exposure import equalize_hist

image_array_hsv[:, :, 2] = equalize_hist(image_array_hsv[:, :, 2])
print(f"V channel after equalization:\n{image_array_hsv[:, :, 2]}")

# Transform HSV to RGB
image_array_rgb = hsv2rgb(image_array_hsv)

# Show the image
image = Image.fromarray((image_array_rgb * 255).astype(np.uint8))
image.show()

# Compute Power

In [ ]:
def compute_power(image):
    img = image.astype(np.float64)
    gamma = 0.7755
    w0 = 1.48169521e-6
    w_r = 2.13636845e-7
    w_g = 1.77746705e-7
    w_b = 2.14348309e-7
    
    r_pow = w_r * np.power(img[:, :, 0], gamma)
    g_pow = w_g * np.power(img[:, :, 1], gamma)
    b_pow = w_b * np.power(img[:, :, 2], gamma)
    
    return w0 + np.sum(r_pow + g_pow + b_pow)

# Compute Distortion

In [ ]:
def compute_distortion(original_img,modified_img):
    if original_img.shape != modified_img.shape:
        original_img = original_img[:,:,:3]
        modified_img = modified_img[:,:,:3]

    lab_orig = color.rgb2lab(original_img)
    lab_mod = color.rgb2lab(modified_img)
    
    diff = lab_orig - lab_mod
    dist_pixel = np.sqrt(np.sum(np.square(diff), axis=2))
    total_error = np.sum(dist_pixel)
    
    h, w, _ = original_img.shape
    max_dist = np.sqrt(100**2 + 255**2 + 255**2)
    
    return (total_error / (h * w * max_dist)) * 100


In [ ]:


power_result_original=compute_power(np.array(original_image))
power_result_v1=compute_power(np.array(image_v1_darker))
distortion_result=compute_distortion(np.array(original_image),np.array(image_v1_darker))

print(power_result_original)
print(power_result_v1)
print(distortion_result)

In [ ]:
#professor provides
from typing import Tuple

def displayed_image(
        i_cell: np.ndarray,
        vdd: float,
        p1: float = 4.251e-5,
        p2: float = -3.029e-4,
        p3: float = 3.024e-5,
        orig_vdd: float = 15,
        ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Display an image on the OLED display taking into account the effect of DVS.

    :param i_cell: An array of the currents drawn by each pixel of the display.
    :param vdd: The new voltage of the display.
    """
    i_cell_max = (p1 * vdd * 1) + (p2 * 1) + p3
    image_rgb_max = (i_cell_max - p3) / (p1 * orig_vdd + p2) * 255
    out = np.round((i_cell - p3) / (p1 * orig_vdd + p2) * 255)
    original_image = out.copy()

    # Clip the values exceeding `i_cell_max` to `image_rgb_max`
    out[i_cell > i_cell_max] = image_rgb_max

    return original_image.astype(np.uint8), out.astype(np.uint8)

In [ ]:
#professor provides
from scipy.io import loadmat

# Load the .mat file
mat_data = loadmat('sample_cell_current.mat')["I_cell_sample"]

# Inspect the loaded data
print(f"Shape: {mat_data.shape}")
print(f"Red channel:\n{mat_data[:, :, 0]}")
print(f"Green channel:\n{mat_data[:, :, 1]}")
print(f"Blue channel:\n{mat_data[:, :, 2]}")

image_array_orig, image_array_w_dvs = displayed_image(mat_data, 10)
image_orig = Image.fromarray(image_array_orig)
image_orig.show()
image_w_dvs = Image.fromarray(image_array_w_dvs)
image_w_dvs.show()


In [ ]:
def estimate_power_part1(img):
    return compute_power(img) 

In [ ]:
def apply_hungry_blue(img, k):
    img_mod = img.astype(np.int16) #default uint8[0,255] underflow
    img_mod[:, :, 2] = img_mod[:, :, 2] - k
    img_mod = np.clip(img_mod, 0, 255) 
    return img_mod.astype(np.uint8)

In [ ]:
#load images function
def load_images_from_folder(folder_path, limit=50):
    images_list = []
    
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    
    try:
        filenames = os.listdir(folder_path)
    except FileNotFoundError:
        print(f"Error: folder '{folder_path}'")
        return []

    filenames.sort()

    count = 0
    print(f"Loading images from {folder_path}...")

    for filename in filenames:
        if count >= limit:
            break
        
        if filename.lower().endswith(valid_extensions):
            file_path = os.path.join(folder_path, filename)
            
            try:
                with Image.open(file_path) as img:
                    img_rgb = img.convert('RGB')
                    
                    img_array = np.array(img_rgb)
                    
                    images_list.append(img_array)
                    count += 1
                    
            except Exception as e:
                print(f"Failed to load image {filename}: {e}")

    print(f"Loaded {len(images_list)} images successfully.")
    return images_list

In [ ]:
# Load images from the specified folder
# 3 groups of images: 50 test images, 5 screen images, and all together
my_folder_path="./test_images"
images = load_images_from_folder(my_folder_path, limit=50)
screen_folder_path="./myscreen"
screen_images = load_images_from_folder(screen_folder_path, limit=5)

all_images = images + screen_images


In [ ]:
#hungry blue pareto curves
#compare the two groups of images (natural vs. screenshots) in terms of distortion and power saving for different k values
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Define the k value range to test
k_values = range(0, 256, 10) 

def run_hungry_blue_experiment(image_list):
    """General function: run Hungry Blue on a set of images and return average statistics"""
    avg_results = []
    for k in k_values:
        savings = []
        distortions = []
        for img in image_list:
            # 1. Compute original power
            p_orig = estimate_power_part1(img)
            # 2. Apply transformation
            img_new = apply_hungry_blue(img, k)
            # 3. Compute new power and distortion
            p_new = estimate_power_part1(img_new)
            dist = compute_distortion(img, img_new)
            # 4. Record energy saving rate
            savings.append((p_orig - p_new) / p_orig * 100 if p_orig > 0 else 0)
            distortions.append(dist)
        # Return average (distortion, saving) for this k value
        avg_results.append((np.mean(distortions), np.mean(savings)))
    return avg_results

# --- Run experiments on each group separately ---
print("Processing natural image group (50 images)...")
results_natural = run_hungry_blue_experiment(images)

print("Processing screenshot group (5 images)...")
results_screenshots = run_hungry_blue_experiment(screen_images)

# --- Visualize comparison ---
plt.figure(figsize=(12, 7))

# --- Plot natural image group ---
x_nat = [r[0] for r in results_natural]
y_nat = [r[1] for r in results_natural]
plt.plot(x_nat, y_nat, 'b-o', label='Natural Images')

# Label points for natural image group
for i, k in enumerate(k_values):
    plt.text(x_nat[i], y_nat[i] + 0.5, f'k={k}', color='blue', fontsize=9, ha='center')

# --- Plot screenshot group ---
x_scr = [r[0] for r in results_screenshots]
y_scr = [r[1] for r in results_screenshots]
plt.plot(x_scr, y_scr, 'r--s', label='Screenshots')

# Label points for screenshot group
for i, k in enumerate(k_values):
    plt.text(x_scr[i], y_scr[i] + 0.5, f'k={k}', color='red', fontsize=9, ha='center')

plt.title('Hungry Blue: Impact of Parameter k on Natural vs. Screenshots')
plt.xlabel('Average Distortion (%)')
plt.ylabel('Average Power Saving (%)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()
plt.show()

In [ ]:
# further compare the two groups by separating screenshots into white and black backgrounds

white_screens = [screen_images[0], screen_images[2]]
black_screens = [screen_images[1], screen_images[3], screen_images[4]]

print("Computing white background screenshots...")
results_white = run_hungry_blue_experiment(white_screens)

print("Computing black background screenshots...")
results_black = run_hungry_blue_experiment(black_screens)

# Plot comparison
plt.figure(figsize=(10, 6))
plt.plot([r[0] for r in results_white], [r[1] for r in results_white], 'r-o', label='White Background (Light Mode)')
plt.plot([r[0] for r in results_black], [r[1] for r in results_black], 'k-s', label='Black Background (Dark Mode)')
plt.plot([r[0] for r in results_natural], [r[1] for r in results_natural], 'b--', label='Natural Images')

plt.title('Impact of Background Color on Power Saving (Hungry Blue)')
plt.xlabel('Average Distortion (%)')
plt.ylabel('Average Power Saving (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Build per-image detailed data for Hungry Blue, stored in detailed_data
detailed_data = {}

for k in k_values:
    current_k_stats = []
    for idx, img in enumerate(all_images):
        p_orig = estimate_power_part1(img)
        img_new = apply_hungry_blue(img, k)
        p_new = estimate_power_part1(img_new)
        saving = (p_orig - p_new) / p_orig * 100 if p_orig > 0 else 0
        dist = compute_distortion(img, img_new)
        current_k_stats.append({'idx': idx, 'saving': saving, 'dist': dist})
    detailed_data[k] = current_k_stats

print(f"detailed_data built: {len(detailed_data)} k values, {len(next(iter(detailed_data.values())))} images each")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_simplified_analysis(k, stats, images):
    # Filter out-of-range idx
    valid_stats = [s for s in stats if s['idx'] < len(images)]
    
    if not valid_stats:
        print("Error: no idx in the input data matches the current image list!")
        return

    # Select extreme values as representative examples
    min_dist = min(valid_stats, key=lambda x: x['dist'])
    max_dist = max(valid_stats, key=lambda x: x['dist'])
    cases = [('MIN Distortion', min_dist, '#2ca02c'), 
             ('MAX Distortion', max_dist, '#ff7f0e')]

    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle(f'Hungry Blue Distortion Analysis: k = {k} ', fontsize=22, fontweight='bold', y=0.98)

    col_titles = ['Original Image', 'Processed (−B)', 'Power Contrib (RGB Mean)']
    for ci, ct in enumerate(col_titles):
        axes[0, ci].set_title(ct, fontsize=16, fontweight='bold', pad=15)

    for row, (label, stat, color_code) in enumerate(cases):
        idx = stat['idx']
        img_orig = images[idx]
        
        # Apply algorithm (ensure the function is defined in your environment)
        img_proc = apply_hungry_blue(img_orig, k) 

        # Compute absolute power
        p_orig_mw = compute_power(img_orig)*1000  # convert to mW
        p_proc_mw = compute_power(img_proc)*1000  # convert to mW
        p_saved_mw = p_orig_mw - p_proc_mw

        # Left data label: add absolute power comparison
        ax_label = axes[row, 0]
        label_str = (f"{label} (Img #{idx})\n"
                     f"────────────\n"
                     f"Orig Pwr: {p_orig_mw:.1f} mW\n"
                     f"Saved: {p_saved_mw:.1f} mW\n"
                     f"────────────\n"
                     f"Sv Ratio: {stat['saving']:.1f}%\n"
                     f"Dst: {stat['dist']:.2f}%")
        
        # Shift text left slightly to avoid overlapping with images
        ax_label.text(-0.55, 0.5, label_str, transform=ax_label.transAxes, 
                      fontsize=13, fontweight='bold', color=color_code, ha='center', va='center',
                      bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=5))

        # 1. Original image
        axes[row, 0].imshow(img_orig)
        axes[row, 0].axis('off')

        # 2. Processed image
        axes[row, 1].imshow(img_proc)
        axes[row, 1].axis('off')

        # 3. Power ratio bar chart 
        ax_bar = axes[row, 2]
        orig_means = [img_orig[:,:,0].mean(), img_orig[:,:,1].mean(), img_orig[:,:,2].mean()]
        proc_means = [img_proc[:,:,0].mean(), img_proc[:,:,1].mean(), img_proc[:,:,2].mean()]
        
        x = np.arange(3)
        width = 0.35
        ax_bar.bar(x - width/2, orig_means, width, label='Orig', color=['#ff9999', '#99ff99', '#9999ff'], edgecolor='black')
        ax_bar.bar(x + width/2, proc_means, width, label='Proc', color=['red', 'green', 'blue'], edgecolor='black')
        
        ax_bar.set_xticks(x)
        ax_bar.set_xticklabels(['Red', 'Green', 'Blue'], fontweight='bold', fontsize=12)
        ax_bar.legend(loc='upper right', fontsize=10)
        ax_bar.grid(axis='y', alpha=0.3)
        ax_bar.set_ylabel('Mean Pixel Value (Power Proxy)', fontsize=12)
        ax_bar.set_ylim(0, max(max(orig_means), max(proc_means)) * 1.2)

    plt.subplots_adjust(left=0.18, right=0.95, wspace=0.15, hspace=0.3)
    plt.show()

# Example call:
if 120 in detailed_data:
    plot_simplified_analysis(120, detailed_data[120], images)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_saving_analysis(k, stats, images):
    """
    Visualize energy saving analysis: fix text alignment, adapt to different aspect ratios
    """
    valid_stats = [s for s in stats if s['idx'] < len(all_images)]
    if not valid_stats:
        print("Error: data mismatch")
        return

    # Select extreme values of energy saving rate
    min_stat = min(valid_stats, key=lambda x: x['saving'])
    max_stat = max(valid_stats, key=lambda x: x['saving'])
    cases = [('MIN Power Saving', min_stat, '#d62728'), 
             ('MAX Power Saving', max_stat, '#1f77b4')]

    fig, axes = plt.subplots(2, 3, figsize=(22, 12))
    fig.suptitle(f'Hungry Blue Power Saving Analysis: k = {k}', fontsize=26, fontweight='bold', y=0.98)

    col_titles = ['Original Image', 'Processed (−B)', 'Power Contrib (RGB Mean)']
    for ci, ct in enumerate(col_titles):
        axes[0, ci].set_title(ct, fontsize=18, fontweight='bold', pad=25)

    for row, (label, stat, color_code) in enumerate(cases):
        idx = stat['idx']
        img_orig = all_images[idx]
        # Note: assumes apply_hungry_blue is defined externally
        img_proc = apply_hungry_blue(img_orig, k) 

        # Compute absolute power data
        p_orig_mw = compute_power(img_orig)*1000  # convert to mW
        p_proc_mw = compute_power(img_proc)*1000  # convert to mW
        p_saved_mw = p_orig_mw - p_proc_mw

        # Annotate text info (use monospace font to align numbers)
        label_str = (f" {label}\n"
                     f" (Img #{idx})\n"
                     f" ────────────\n"
                     f" Orig Pwr: {p_orig_mw:>7.1f} mW\n"
                     f" Saved:    {p_saved_mw:>7.1f} mW\n"
                     f" ────────────\n"
                     f" Sv Ratio: {stat['saving']:>7.1f}%\n"
                     f" Dst:      {stat['dist']:>7.2f}%")
        
        # ==================== [Key: use Figure coordinates for alignment] ====================
        ax_label = axes[row, 0]
        # Get subplot position on canvas, compute vertical center
        pos = ax_label.get_position()
        v_center = (pos.y0 + pos.y1) / 2
        
        # Use fig.text to lock x=0.03 (leftmost 3% of canvas)
        # family='monospace' ensures numbers align vertically
        fig.text(0.03, v_center, label_str, 
                 fontsize=15, fontweight='bold', color=color_code, 
                 family='monospace',
                 ha='left', va='center', linespacing=1.8,
                 bbox=dict(facecolor='white', alpha=0.95, edgecolor=color_code, lw=2.5, pad=15))

        # Draw original and processed images
        axes[row, 0].imshow(img_orig)
        axes[row, 0].axis('off')
        axes[row, 1].imshow(img_proc)
        axes[row, 1].axis('off')

        # Draw RGB channel mean bar chart
        ax_bar = axes[row, 2]
        orig_means = [img_orig[:,:,i].mean() for i in range(3)]
        proc_means = [img_proc[:,:,i].mean() for i in range(3)]
        
        x_indices = np.arange(3)
        width = 0.35
        ax_bar.bar(x_indices - width/2, orig_means, width, label='Orig', color=['#ff9999', '#99ff99', '#9999ff'], edgecolor='black', alpha=0.7)
        ax_bar.bar(x_indices + width/2, proc_means, width, label='Proc', color=['red', 'green', 'blue'], edgecolor='black')
        
        ax_bar.set_xticks(x_indices)
        ax_bar.set_xticklabels(['Red', 'Green', 'Blue'], fontweight='bold', fontsize=13)
        ax_bar.set_ylabel('Mean Pixel Value (Power Proxy)', fontsize=12)
        ax_bar.grid(axis='y', alpha=0.3)
        ax_bar.set_ylim(0, max(orig_means) * 1.3)
        ax_bar.legend(loc='upper right', fontsize=11)

    plt.subplots_adjust(left=0.23, right=0.96, wspace=0.35, hspace=0.4)
    plt.show()

# Example call
if 120 in detailed_data:
    plot_saving_analysis(120, detailed_data[120], images)

In [ ]:
def apply_contrast_brightness(img, scale_v):
    # Convert to HSV (skimage returns 0-1 float)
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = hsv[:, :, 2] * scale_v # adjust V channel
    hsv[:, :, 2] = np.clip(hsv[:, :, 2], 0, 1) # clip to 0-1
    # Convert back to RGB and scale to 0-255
    return (color.hsv2rgb(hsv) * 255).astype(np.uint8)

In [ ]:
#strategy 2: Brightness Scaling 
#table 50 natural images or total images
import numpy as np
import pandas as pd

# Store averages for Pareto curve
results_contrast = []
# Store detailed data for statistics table and representative images
detailed_data_contrast = {}

c_values = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1] 

print("Running Strategy 2: Brightness Scaling (detailed stats)...")

for c in c_values:
    current_c_stats = []
    
    for idx, img in enumerate(all_images):
        p_orig = estimate_power_part1(img)
        img_new = apply_contrast_brightness(img, c)
        p_new = estimate_power_part1(img_new)
        
        if p_orig > 0:
            saving = (p_orig - p_new) / p_orig * 100
        else:
            saving = 0
            
        dist = compute_distortion(img, img_new)
        
        # Save single image result
        current_c_stats.append({
            "idx": idx,
            "saving": saving,
            "dist": dist
        })
        
    # Compute averages (for plotting)
    all_savings = [x['saving'] for x in current_c_stats]
    all_dists = [x['dist'] for x in current_c_stats]
    
    results_contrast.append((np.mean(all_dists), np.mean(all_savings)))
    
    # Save detailed data to dict
    detailed_data_contrast[c] = current_c_stats

print("Strategy 2 computation done!")

# --- Show Strategy 2 statistics table ---
stats_list_cont = []
for c, stats in detailed_data_contrast.items():
    savings = [s['saving'] for s in stats]
    dists = [s['dist'] for s in stats]
    stats_list_cont.append({
        "Parameter (c)": c,
        "Avg Saving %": np.mean(savings),
        "Min Saving %": np.min(savings), # Min (as required)
        "Max Saving %": np.max(savings), # Max (as required)
        "Avg Distortion": np.mean(dists),
        "Min Distortion": np.min(dists),
        "Max Distortion": np.max(dists)
    })

print("\n--- Summary Table for Brightness Scaling ---")
display(pd.DataFrame(stats_list_cont).round(2))

In [ ]:
x_cont = [r[0] for r in results_contrast]
y_cont = [r[1] for r in results_contrast]
plt.plot(x_cont, y_cont, 'g-s', label='Brightness Scaling (Reduce V)')
for x, y, c in zip(x_cont, y_cont, c_values):
    if x < 10:
        # y+0.5 places text slightly above the point
        plt.text(x + 0.1, y + 0.5, f'c={c:.1f}', color='green', fontsize=9)
plt.title('Pareto Curve: Power Saving vs. Image Distortion')
plt.xlabel('Average Distortion (%)')
plt.ylabel('Average Power Saving (%)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()
plt.xlim(0, 11) 
    
plt.show()

In [ ]:
# Extract data
c_vals = [r['Parameter (c)'] for r in stats_list_cont]
avg_savings = [r['Avg Saving %'] for r in stats_list_cont]
avg_dists = [r['Avg Distortion'] for r in stats_list_cont]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: c vs Saving (reveals Gamma effect)
ax1.plot(c_vals, avg_savings, 'ro-', label='Saving')
ax1.set_xlabel('Parameter c (Brightness Factor)')
ax1.set_ylabel('Average Power Saving (%)')
ax1.set_title('c vs. Power Saving\n')
ax1.invert_xaxis() # display c from 1.0 to 0.1
ax1.grid(True, linestyle='--', alpha=0.6)

# Plot 2: c vs Distortion (reveals Lab space nonlinearity)
ax2.plot(c_vals, avg_dists, 'bo-', label='Distortion')
ax2.set_xlabel('Parameter c (Brightness Factor)')
ax2.set_ylabel('Average Distortion (%)')
ax2.set_title('c vs. Distortion\n')
ax2.invert_xaxis()
ax2.grid(True, linestyle='--', alpha=0.6)

plt.show()

In [ ]:
def plot_full_set_energy_breakdown(target_c, group_indices, images):
    """
    Analyze energy breakdown for all 50 images, sorted by distortion
    """
    results = []
    for idx in group_indices:
        orig = images[idx]
        proc = apply_contrast_brightness(orig, target_c)
        lab_o, lab_p = color.rgb2lab(orig), color.rgb2lab(proc)
        
        diff_sq = (lab_o - lab_p)**2
        msd_l, msd_a, msd_b = np.mean(diff_sq[:,:,0]), np.mean(diff_sq[:,:,1]), np.mean(diff_sq[:,:,2])
        total_msd = msd_l + msd_a + msd_b
        
        results.append({
            'idx': idx,
            'total_dist': np.sqrt(total_msd),
            'l_pct': msd_l / total_msd * 100,
            'a_pct': msd_a / total_msd * 100,
            'b_pct': msd_b / total_msd * 100
        })

    # Sort by total distortion descending to see the trend
    results_sorted = sorted(results, key=lambda x: x['total_dist'], reverse=True)

    # Plot
    plt.figure(figsize=(18, 7))
    indices = np.arange(len(results_sorted))
    l_vals = [r['l_pct'] for r in results_sorted]
    a_vals = [r['a_pct'] for r in results_sorted]
    b_vals = [r['b_pct'] for r in results_sorted]
    
    plt.bar(indices, l_vals, color='gold', label='$\Delta L^2$ (Brightness)')
    plt.bar(indices, a_vals, bottom=l_vals, color='green', label='$\Delta a^2$ (Chroma-a)')
    plt.bar(indices, b_vals, bottom=np.array(l_vals)+np.array(a_vals), color='blue', label='$\Delta b^2$ (Chroma-b)')

    plt.title(f"Energy Contribution Across All 50 Natural Images C=0.7", fontsize=15)
    plt.xlabel("Images (Sorted from High Distortion to Low Distortion)", fontsize=12)
    plt.ylabel("Contribution (%)", fontsize=12)
    plt.xticks(indices, [r['idx'] for r in results_sorted], rotation=90, fontsize=8)
    plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1))
    plt.grid(axis='y', linestyle=':', alpha=0.5)
    plt.tight_layout()
    plt.show()

# Execute
plot_full_set_energy_breakdown(0.7, range(0, 50), all_images)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import color

def pad_image_to_center(img, target_h, target_w, bg_color=(1, 1, 1)):
    """
    Helper: pad image to target size (target_h, target_w),
    center the original, fill background with bg_color (default white).
    """
    h, w, c = img.shape
    
    # Create solid color background canvas
    # Assumes input is 0-1 float; adjust if 0-255 uint8
    if img.dtype == np.uint8:
        bg_val = [int(c * 255) for c in bg_color]
        canvas = np.full((target_h, target_w, c), bg_val, dtype=np.uint8)
    else:
        canvas = np.full((target_h, target_w, c), bg_color, dtype=np.float64)
    
    # Compute center position
    y_off = (target_h - h) // 2
    x_off = (target_w - w) // 2
    
    # Place original image
    canvas[y_off:y_off+h, x_off:x_off+w, :] = img
    return canvas

def analyze_distortion_aligned(target_c, group_indices):
    """
    Alignment-optimized version:
    1. Compute Max/Min data.
    2. Pad images to the same size for alignment.
    3. Display detailed data.
    """
    
    # --- 1. Data selection and preparation ---
    all_stats = detailed_data_contrast[target_c]
    group_stats = [item for item in all_stats if item['idx'] in group_indices]
    
    if not group_stats: return

    # Find Max and Min
    max_item = sorted(group_stats, key=lambda x: x['dist'], reverse=True)[0]
    min_item = sorted(group_stats, key=lambda x: x['dist'], reverse=False)[0]
    
    cases = [
        ("MAX Distortion (Worst)", max_item, '#7b241c'), 
        ("MIN Distortion (Best)",  min_item, '#1b4f72') 
    ]
    
    # --- 2. Preprocessing: unify image size for alignment ---
    # Extract images first
    imgs_data = []
    for _, item, _ in cases:
        idx = item['idx']
        orig = all_images[idx] # original image
        # Note: assumes apply_contrast_brightness returns the same size as original
        proc = apply_contrast_brightness(orig, target_c) 
        imgs_data.append({'orig': orig, 'proc': proc, 'idx': idx})

    # Compute max width and height
    max_h = max(d['orig'].shape[0] for d in imgs_data)
    max_w = max(d['orig'].shape[1] for d in imgs_data)

    # --- 3. Plot initialization ---
    # Use width_ratios to control column width: images 1 share, text box 1.2
    fig, axes = plt.subplots(2, 3, figsize=(18, 12), 
                             gridspec_kw={'width_ratios': [1, 1, 1.2]})
    
    fig.suptitle(f"MAX and MIN distortion images| c={target_c}", 
                 fontsize=20, fontweight='bold', y=0.96)

    for row_idx, (title_prefix, item, theme_color) in enumerate(cases):
        # Retrieve the prepared data
        current_data = imgs_data[row_idx]
        orig_raw = current_data['orig']
        proc_raw = current_data['proc']
        img_idx = current_data['idx']
        distortion_val = item['dist']

        # --- A. Core data computation (use raw original image, not padded) ---
        # 1. RGB
        avg_rgb_255 = np.mean(orig_raw, axis=(0, 1))
        r, g, b = avg_rgb_255 / 255.0
        
        # 2. Lab
        lab_orig = color.rgb2lab(orig_raw)
        l_o, a_o, b_o = np.mean(lab_orig, axis=(0, 1))
        
        lab_proc = color.rgb2lab(proc_raw)
        l_p, a_p, b_p = np.mean(lab_proc, axis=(0, 1))
        
        delta_L = l_p - l_o

        # --- B. Visual alignment (Padding) ---
        # Pad image to max_h, max_w with white background to blend with canvas
        orig_display = pad_image_to_center(orig_raw, max_h, max_w, bg_color=(1,1,1))
        proc_display = pad_image_to_center(proc_raw, max_h, max_w, bg_color=(1,1,1))

        # --- C. Plotting ---
        
        # [Column 1] Original
        ax_orig = axes[row_idx, 0]
        ax_orig.imshow(orig_display)
        ax_orig.set_title(f"Original | ID: {img_idx}", fontsize=14, fontweight='bold')
        ax_orig.axis('off')
        
        # [Column 2] Processed
        ax_proc = axes[row_idx, 1]
        ax_proc.imshow(proc_display)
        ax_proc.set_title(f"Processed (c={target_c})", fontsize=14, fontweight='bold', color=theme_color)
        ax_proc.axis('off')
        
        # [Column 3] Info Board
        ax_text = axes[row_idx, 2]
        ax_text.axis('off')
        
        info_text = (
            f"CASE: {title_prefix}\n"
            f"Distortion: {distortion_val:.4f}\n"
            f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n"
            f"[1] Original RGB (Normalized):\n"
            f"    R={r:.3f}  G={g:.3f}  B={b:.3f}\n\n"
            
            f"[2] Lab Statistics (Avg):\n"
            f"    Orig: L={l_o:.1f}  (a={a_o:.1f}, b={b_o:.1f})\n"
            f"    Proc: L={l_p:.1f}  (a={a_p:.1f}, b={b_p:.1f})\n\n"
            
            f"[3] Lightness Shift (ΔL):\n"
            f"    ΔL = {delta_L:+.2f} ({'Brighter' if delta_L>0 else 'Darker'})"
        )
        
        # Vertically center the text box (valign='center')
        ax_text.text(0.05, 0.5, info_text, transform=ax_text.transAxes, 
                     fontsize=13, family='monospace', verticalalignment='center',
                     bbox=dict(boxstyle='round,pad=0.8', facecolor='#fdfdfd', 
                               edgecolor=theme_color, linewidth=2))

    # Adjust subplot spacing
    plt.tight_layout(rect=[0, 0, 1, 0.94])
    # Slightly increase row spacing to avoid crowding
    plt.subplots_adjust(hspace=0.15, wspace=0.05)
    plt.show()

# --- Execute ---
TARGET_C = 0.7
NATURAL_INDICES = range(0, len(all_images))

print(f"Generating aligned combined analysis plot (Target c={TARGET_C})...")
analyze_distortion_aligned(TARGET_C, NATURAL_INDICES)

In [ ]:
# Strategy: Pure Histogram Equalization (No Alpha)
# Strategy: Pure Histogram Equalization (No Alpha)
from skimage import color, exposure
import numpy as np

print("Running Standard Histogram Equalization (Baseline)...")

# Store results
results_pure_hist = []

for idx, img in enumerate(images):
    # 1. Compute original power
    p_orig = estimate_power_part1(img)
    
    # 2. Convert to HSV space
    # skimage.color.rgb2hsv converts image to float64 (range 0-1)
    hsv = color.rgb2hsv(img)
    
    # 3. Apply histogram equalization to V channel (brightness)
    # exposure.equalize_hist output is also float64 (range 0-1)
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    
    # 4. Convert back to RGB and restore to 0-255 (uint8)
    img_eq = (color.hsv2rgb(hsv) * 255).astype(np.uint8)
    
    # 5. Compute new power
    p_new = estimate_power_part1(img_eq)
    
    # 6. Compute metrics
    if p_orig > 0:
        saving = (p_orig - p_new) / p_orig * 100
    else:
        saving = 0
        
    dist = compute_distortion(img, img_eq)
    
    results_pure_hist.append({
        "image_idx": idx,
        "saving": saving,
        "dist": dist
    })

# --- Summary statistics ---
savings = [r['saving'] for r in results_pure_hist]
dists = [r['dist'] for r in results_pure_hist]

print("\n--- Summary: Standard Histogram Equalization ---")
print("Power Saving Statistics:")
print(f"  Average: {np.mean(savings):.2f}%")
print(f"  Min:     {np.min(savings):.2f}% (negative means power increase)")
print(f"  Max:     {np.max(savings):.2f}%")  # added

print("\nDistortion Statistics:")
print(f"  Average: {np.mean(dists):.2f}%")
print(f"  Min:     {np.min(dists):.2f}%")    # added
print(f"  Max:     {np.max(dists):.2f}%")

In [ ]:
# Analyze baseline (Part 1: Pure Histogram Equalization Results)
import matplotlib.pyplot as plt
import numpy as np

# Assumes results_pure_hist was computed in a previous cell
# results_pure_hist = [{'image_idx': 0, 'saving': 1.2, 'dist': 2.5}, ...]

# 1. Find indices of key images
# Sort by saving (Saving %)
sorted_by_saving = sorted(results_pure_hist, key=lambda x: x['saving'])
min_saving_idx = sorted_by_saving[0]['image_idx']      # Most power increase (Worst Case)
max_saving_idx = sorted_by_saving[-1]['image_idx']     # Most power saving (Best Case)

# Sort by distortion (Distortion %)
sorted_by_dist = sorted(results_pure_hist, key=lambda x: x['dist'])
min_dist_idx = sorted_by_dist[0]['image_idx']          # Min distortion (Min Distortion) - NEW!
max_dist_idx = sorted_by_dist[-1]['image_idx']         # Max distortion (Max Distortion)

# Print key indices and values
print(f"Min Saving Image Index: {min_saving_idx} (Saving: {sorted_by_saving[0]['saving']:.2f}%)")
print(f"Max Saving Image Index: {max_saving_idx} (Saving: {sorted_by_saving[-1]['saving']:.2f}%)")
print(f"Min Distortion Image Index: {min_dist_idx} (Dist: {sorted_by_dist[0]['dist']:.2f}%)") # NEW!
print(f"Max Distortion Image Index: {max_dist_idx} (Dist: {sorted_by_dist[-1]['dist']:.2f}%)")

# 2. Define plot function (including image and histogram)
def plot_comparison(idx, title_prefix):
    img_orig = images[idx]
    
    # Redo transformation (to get the processed image)
    hsv = color.rgb2hsv(img_orig)
    # Apply histogram equalization to V channel (brightness)
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img_trans = (color.hsv2rgb(hsv) * 255).astype(np.uint8)
    
    # Compute specific values for this image
    p_orig = estimate_power_part1(img_orig)
    p_new = estimate_power_part1(img_trans)
    saving = (p_orig - p_new) / p_orig * 100 if p_orig > 0 else 0
    dist = compute_distortion(img_orig, img_trans)

    # --- Plot ---
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle(f"{title_prefix} (Image {idx})\nSaving: {saving:.2f}%, Distortion: {dist:.2f}%", fontsize=14)
    
    # Original image
    axes[0, 0].imshow(img_orig)
    axes[0, 0].set_title("Original Image")
    axes[0, 0].axis('off')
    
    # Transformed image
    axes[0, 1].imshow(img_trans)
    axes[0, 1].set_title("Equalized Image")
    axes[0, 1].axis('off')
    
    # Original image histogram (RGB)
    axes[1, 0].set_title("Original Histogram (Pixel Distribution)")
    for i, col in enumerate(['r', 'g', 'b']):
        hist, bins = np.histogram(img_orig[:, :, i], bins=256, range=(0, 256))
        axes[1, 0].plot(bins[:-1], hist, color=col, alpha=0.7)
    axes[1, 0].set_xlim([0, 256])
    axes[1, 0].grid(alpha=0.3)
    
    # Transformed histogram (RGB)
    axes[1, 1].set_title("Equalized Histogram (Pixel Distribution)")
    for i, col in enumerate(['r', 'g', 'b']):
        hist, bins = np.histogram(img_trans[:, :, i], bins=256, range=(0, 256))
        axes[1, 1].plot(bins[:-1], hist, color=col, alpha=0.7)
    axes[1, 1].set_xlim([0, 256])
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 3. Execute plotting
print("\n--- Case 1: Worst Power Saving (Increased Power) ---")
plot_comparison(min_saving_idx, "Worst Case Power Saving")

print("\n--- Case 2: Best Power Saving ---")
plot_comparison(max_saving_idx, "Best Case Power Saving")

print("\n--- Case 3: Min Distortion (Subtle Change) ---") # NEW!
plot_comparison(min_dist_idx, "Best Case Distortion (Min)")

print("\n--- Case 4: Max Distortion ---")
plot_comparison(max_dist_idx, "Worst Case Distortion (Max)")

In [ ]:
# Strategy 3: Weighted Histogram Equalization
from skimage import color, exposure
import pandas as pd
import numpy as np

# Assumes estimate_power_part1 and compute_distortion are already defined
# Assumes images is your image list

results_hist = []
detailed_data_hist = {}

# Alpha represents "weight of equalized image". 0=original, 1=full equalization
alphas = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

print("Running Strategy 3: Histogram Equalization (optimized)...")

# For convenience, transpose data structure: group by alpha
alpha_stats_storage = {a: [] for a in alphas}

for idx, img in enumerate(images):
    # 1. Base power
    p_orig = estimate_power_part1(img)
    
    # 2. [Optimization] compute full equalized image once outside the loop (img_eq)
    # Convert to HSV
    hsv = color.rgb2hsv(img)
    # Equalize V channel (hsv[:, :, 2])
    # Note: exposure.equalize_hist returns float 0-1
    hsv_eq = hsv.copy()
    hsv_eq[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    # Convert back to RGB and restore to 0-255 (uint8)
    img_eq = (color.hsv2rgb(hsv_eq) * 255).astype(np.uint8)
    
    # 3. Iterate over Alpha for blending
    for alpha in alphas:
        if alpha == 0:
            img_mixed = img
        elif alpha == 1.0:
            img_mixed = img_eq
        else:
            # Linear blend: (1-a)*original + a*equalized
            # Must convert to float for computation, then back to uint8
            img_mixed = (img.astype(float) * (1 - alpha) + img_eq.astype(float) * alpha)
            img_mixed = np.clip(img_mixed, 0, 255).astype(np.uint8)
        
        # Compute power and distortion
        p_new = estimate_power_part1(img_mixed)
        
        if p_orig > 0:
            saving = (p_orig - p_new) / p_orig * 100
        else:
            saving = 0
            
        dist = compute_distortion(img, img_mixed) # use LAB distance function defined earlier
        
        # Store in the list for the corresponding alpha
        alpha_stats_storage[alpha].append({
            "image_idx": idx,
            "saving": saving,
            "dist": dist
        })

print("Computation done! Summarizing data...")

# --- Summarize data for display and plotting ---
stats_list_hist = []

for alpha in alphas:
    stats = alpha_stats_storage[alpha]
    
    # Extract saving and dist for all images
    savings = [s['saving'] for s in stats]
    dists = [s['dist'] for s in stats]
    
    # Record averages for Pareto curve (Distortion, Saving)
    mean_dist = np.mean(dists)
    mean_saving = np.mean(savings)
    results_hist.append((mean_dist, mean_saving))
    
    # Record detailed statistics table
    stats_list_hist.append({
        "Alpha (Weight)": alpha,
        "Avg Saving %": mean_saving,
        "Min Saving %": np.min(savings),
        "Max Saving %": np.max(savings),
        "Avg Distortion %": mean_dist,
        "Min Distortion %": np.min(dists), # should be 0 (when alpha=0)
        "Max Distortion %": np.max(dists)
    })

print("\n--- Summary Table for Histogram Equalization ---")
df_hist = pd.DataFrame(stats_list_hist)
display(df_hist.round(2))

# Quickly plot Pareto points (if in Jupyter)
import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))
x_val = [r[0] for r in results_hist]
y_val = [r[1] for r in results_hist]
plt.plot(x_val, y_val, marker='o', linestyle='-')
plt.title("Pareto Curve: Histogram Equalization with alpha")
plt.xlabel("Average Distortion (%)")
plt.ylabel("Average Power Saving (%)")
plt.grid(True)
plt.show()

In [ ]:
# Strategy 4: Gamma Correction (Detailed Stats Version)
from skimage import exposure, color
import numpy as np
import pandas as pd

gammas = [1.0, 1.2, 1.4, 1.6, 1.8, 2.0]
stats_list_gamma = []

print("Running Detailed Gamma Correction Analysis...")

for gamma in gammas:
    current_savings = []
    current_dists = []
    
    for img in images:
        p_orig = estimate_power_part1(img)
        
        # Gamma Transform
        hsv = color.rgb2hsv(img)
        hsv[:, :, 2] = exposure.adjust_gamma(hsv[:, :, 2], gamma=gamma)
        img_gamma = (color.hsv2rgb(hsv) * 255).astype(np.uint8)
        
        # Metrics
        p_new = estimate_power_part1(img_gamma)
        saving = (p_orig - p_new) / p_orig * 100 if p_orig > 0 else 0
        dist = compute_distortion(img, img_gamma)
        
        current_savings.append(saving)
        current_dists.append(dist)
    
    # Summary statistics
    stats_list_gamma.append({
        "Gamma": gamma,
        "Avg Saving": np.mean(current_savings),
        "Min Saving": np.min(current_savings),
        "Max Saving": np.max(current_savings),
        "Avg Dist": np.mean(current_dists),
        "Min Dist": np.min(current_dists),
        "Max Dist": np.max(current_dists)
    })

# Display DataFrame (can paste rows into LaTeX table)
df_gamma = pd.DataFrame(stats_list_gamma)
print("\n--- Gamma Correction Detailed Statistics ---")
display(df_gamma.round(2))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage import exposure, color

# 1. Define the list of Gamma values to test
# 1.2 and 1.6 were mentioned; add more points for a smoother curve
gammas = [1.0, 1.2, 1.4, 1.6, 1.8, 2.0]

# Store the final average result for each Gamma setting
summary_results = []

print("Starting Gamma correction strategy analysis...")

# --- Outer loop: iterate over different Gamma values ---
for gamma in gammas:
    
    # Temp list for per-image data under current Gamma
    current_savings = []
    current_dists = []
    
    # --- Inner loop: iterate over each image ---
    for img in images:
        # A. Compute original power
        p_orig = estimate_power_part1(img)
        
        # B. Apply Gamma transformation (core algorithm)
        # Convert to HSV -> adjust V channel -> convert back to RGB
        hsv = color.rgb2hsv(img)
        hsv[:, :, 2] = exposure.adjust_gamma(hsv[:, :, 2], gamma=gamma)
        img_new = (color.hsv2rgb(hsv) * 255).astype(np.uint8)
        
        # C. Compute new metrics
        p_new = estimate_power_part1(img_new)
        
        # Energy saving rate
        if p_orig > 0:
            saving = (p_orig - p_new) / p_orig * 100
        else:
            saving = 0
            
        # Distortion rate
        dist = compute_distortion(img, img_new)
        
        # D. Collect data
        current_savings.append(saving)
        current_dists.append(dist)
    
    # --- Compute average performance for current Gamma ---
    avg_saving = np.mean(current_savings)
    avg_dist = np.mean(current_dists)
    
    # Save to summary list
    summary_results.append({
        "Gamma": gamma,
        "Avg Saving (%)": avg_saving,
        "Avg Distortion (%)": avg_dist
    })
    
    print(f"Done: Gamma = {gamma} -> Avg saving: {avg_saving:.2f}%, Avg distortion: {avg_dist:.2f}%")

# --- 2. Show data table ---
df_summary = pd.DataFrame(summary_results)
print("\n--- Final summary table ---")
display(df_summary.round(2))

# --- 3. Plot Pareto curve ---
plt.figure(figsize=(8, 6))

# Extract X (distortion) and Y (saving)
x_vals = df_summary["Avg Distortion (%)"]
y_vals = df_summary["Avg Saving (%)"]
labels = df_summary["Gamma"]

# Draw lines and points
plt.plot(x_vals, y_vals, marker='o', linestyle='-', linewidth=2, color='tab:blue', label='Gamma Strategy')

# Label each point with its Gamma value
for i, txt in enumerate(labels):
    plt.annotate(f"γ={txt}", 
                 (x_vals[i], y_vals[i]), 
                 textcoords="offset points", 
                 xytext=(0,10), 
                 ha='center')

# Decorate chart
plt.xlabel("Average Distortion (%)")
plt.ylabel("Average Power Saving (%)")
plt.title("Pareto Curve: Gamma Correction Trade-off")
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()

# Show
plt.show()

In [ ]:
# Find extreme images for a specific gamma
import numpy as np
import pandas as pd
from skimage import exposure, color

# Assumes image list is called images and evaluation functions are defined
target_gamma = 1.4
image_details = []

print(f"Analyzing per-image data for Gamma = {target_gamma}...")

# Use enumerate to get the index (i) of each image
for i, img in enumerate(images):
    p_orig = estimate_power_part1(img)
    
    # Gamma Transform
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.adjust_gamma(hsv[:, :, 2], gamma=target_gamma)
    img_gamma = (color.hsv2rgb(hsv) * 255).astype(np.uint8)
    
    # Metrics
    p_new = estimate_power_part1(img_gamma)
    saving = (p_orig - p_new) / p_orig * 100 if p_orig > 0 else 0
    dist = compute_distortion(img, img_gamma)
    
    # Store the index and result for this image
    image_details.append({
        "Image_Index": i,  # use file_names[i] if you have a filename list
        "Saving": saving,
        "Distortion": dist
    })

# Convert to DataFrame for easy extreme value lookup
df_details = pd.DataFrame(image_details)

# --- Core search logic ---
# idxmax() and idxmin() return row index of max/min value in the column
idx_max_dist = df_details['Distortion'].idxmax()
idx_min_dist = df_details['Distortion'].idxmin()
idx_max_saving = df_details['Saving'].idxmax()
idx_min_saving = df_details['Saving'].idxmin()

print("\n--- Extreme image results (Gamma=1.4) ---")
print(f"Max Distortion image index: {df_details.loc[idx_max_dist, 'Image_Index']}, distortion: {df_details.loc[idx_max_dist, 'Distortion']:.2f}")
print(f"Min Distortion image index: {df_details.loc[idx_min_dist, 'Image_Index']}, distortion: {df_details.loc[idx_min_dist, 'Distortion']:.2f}")
print(f"Max Saving image index: {df_details.loc[idx_max_saving, 'Image_Index']}, saving: {df_details.loc[idx_max_saving, 'Saving']:.2f}%")
print(f"Min Saving image index: {df_details.loc[idx_min_saving, 'Image_Index']}, saving: {df_details.loc[idx_min_saving, 'Saving']:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import color, exposure

# --- 1. Helper Functions Definitions ---

def apply_gamma_and_get_hsv(img_rgb, gamma_val):
    """Helper: Applies Gamma to V channel and returns processed RGB and both HSV arrays."""
    hsv_original = color.rgb2hsv(img_rgb)
    hsv_processed = hsv_original.copy()
    
    # Apply Gamma to the V (Value) channel
    hsv_processed[:, :, 2] = exposure.adjust_gamma(hsv_processed[:, :, 2], gamma=gamma_val)
    
    # Convert back to RGB for display (clip to 0-255 and cast to uint8)
    img_processed = np.clip(color.hsv2rgb(hsv_processed) * 255, 0, 255).astype(np.uint8)
    
    # Return both original and processed HSV data for comparison
    return img_processed, hsv_original, hsv_processed

def plot_hsv_histogram(ax, hsv_data, title_text, highlight_v=True):
    """Helper: Plots HSV histograms. Emphasizes V channel if highlight_v is True."""
    labels = ['Hue', 'Saturation', 'Value (V)']
    
    # Emphasize V channel with bold black line
    plot_styles = [
        {'color': 'magenta', 'lw': 1, 'alpha': 0.5}, # Hue style
        {'color': 'cyan', 'lw': 1, 'alpha': 0.5},    # Saturation style
        {'color': 'black', 'lw': 2 if highlight_v else 1.5, 'alpha': 0.9 if highlight_v else 0.7} # Value style
    ]

    for i, style in enumerate(plot_styles):
        hist, bins = np.histogram(hsv_data[:, :, i].flatten(), bins=100, range=[0.0, 1.0])
        ax.plot(bins[:-1], hist, label=labels[i], **style)
    
    ax.set_title(title_text, fontsize=10)
    ax.set_xlim([0.0, 1.0])
    ax.set_xlabel("Normalized Value (0.0-1.0)")
    ax.set_ylabel("Count")
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

# --- 2. Main Visualization Loop ---

target_gamma = 1.4

# Define the four extreme cases using the indices found previously
# Make sure idx_max_dist, idx_min_dist, idx_max_saving, idx_min_saving are in memory
cases_to_analyze = [
    ("Case 1: Max Distortion", idx_max_dist),
    ("Case 2: Min Distortion", idx_min_dist),
    ("Case 3: Max Saving", idx_max_saving),
    ("Case 4: Min Saving", idx_min_saving)
]

print(f"Starting HSV comparison visualization for Gamma={target_gamma} extreme cases...")

for case_name, img_idx in cases_to_analyze:
    # A. Prepare Data
    img_orig = images[img_idx]
    
    # Now unpacking three variables instead of two
    img_proc, hsv_orig, hsv_proc = apply_gamma_and_get_hsv(img_orig, target_gamma)
    
    # B. Re-calculate metrics specific to this image for the main title
    p_orig_val = estimate_power_part1(img_orig)
    p_new_val = estimate_power_part1(img_proc)
    sav_val = (p_orig_val - p_new_val) / p_orig_val * 100 if p_orig_val > 0 else 0
    dist_val = compute_distortion(img_orig, img_proc)

    # C. Create Figure Canvas (1 row, 4 columns)
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    main_title = f"{case_name} [Index: {img_idx}] - Saving: {sav_val:.2f}%, Dist: {dist_val:.2f}"
    fig.suptitle(main_title, fontsize=14, y=1.02)

    # D. Plot Subplots
    
    # Subplot 1: Original Image
    axes[0].imshow(img_orig)
    axes[0].set_title("Original Image")
    axes[0].axis('off')

    # Subplot 2: Original HSV Histogram
    plot_hsv_histogram(axes[1], hsv_orig, "Original HSV Distribution", highlight_v=True)

    # Subplot 3: Processed Image
    axes[2].imshow(img_proc)
    axes[2].set_title(f"Processed (Gamma {target_gamma})")
    axes[2].axis('off')

    # Subplot 4: Processed HSV Histogram
    plot_hsv_histogram(axes[3], hsv_proc, "Processed HSV Distribution", highlight_v=True)

    plt.tight_layout()
    plt.show()

print("Visualization finished.")

The L shift is large for the bird but almost unchanged for the bear, because converting HSV back to RGB also changes little, so the Lab difference between original and processed is small for the bear.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure, color

# 1. Define parameter ranges uniformly (adjust as needed)
k_values = list(range(0, 256, 10))
c_values = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
alphas = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
# Extend Gamma range slightly to find the optimum near the 3% threshold
gammas = [1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0] 

# 2. Define unified test function
def find_best_saving_under_limit(img_list, transform_func, params, strategy_name, dist_limit=3.0):
    best_saving = 0
    best_param = None
    best_dist = 0
    
    print(f"Scanning all parameters for {strategy_name}...")
    for p in params:
        savings = []
        dists = []
        for img in img_list:
            p_orig = estimate_power_part1(img)
            img_new = transform_func(img, p)
            p_new = estimate_power_part1(img_new)
            
            saving = (p_orig - p_new) / p_orig * 100 if p_orig > 0 else 0
            dist = compute_distortion(img, img_new)
            
            savings.append(saving)
            dists.append(dist)
            
        avg_saving = np.mean(savings)
        avg_dist = np.mean(dists)
        
        # Core logic: distortion must be strictly below limit (3.0%)
        if avg_dist < dist_limit:
            # Among valid parameters, find the one with max saving
            if avg_saving > best_saving:
                best_saving = avg_saving
                best_param = p
                best_dist = avg_dist
                
    print(f"  -> Best param: {best_param}, distortion: {best_dist:.2f}%, saving: {best_saving:.2f}%")
    return best_saving, best_param, best_dist

# 3. Wrap transformation functions for each strategy (based on existing code)
def wrapper_hungry_blue(img, k):
    return apply_hungry_blue(img, k)

def wrapper_brightness(img, c):
    return apply_contrast_brightness(img, c)

def wrapper_he(img, alpha):
    hsv = color.rgb2hsv(img)
    hsv_eq = hsv.copy()
    hsv_eq[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img_eq = (color.hsv2rgb(hsv_eq) * 255).astype(np.uint8)
    img_mixed = (img.astype(float) * (1 - alpha) + img_eq.astype(float) * alpha)
    return np.clip(img_mixed, 0, 255).astype(np.uint8)

def wrapper_gamma(img, gamma):
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.adjust_gamma(hsv[:, :, 2], gamma=gamma)
    return (color.hsv2rgb(hsv) * 255).astype(np.uint8)

# 4. Run evaluation (on 50 natural images)
print("=== Finding max Saving with Distortion < 3% ===")
DIST_LIMIT = 3.0
dataset_to_test = images 

res_hb = find_best_saving_under_limit(dataset_to_test, wrapper_hungry_blue, k_values, "Hungry Blue", DIST_LIMIT)
res_bs = find_best_saving_under_limit(dataset_to_test, wrapper_brightness, c_values, "Brightness Scaling", DIST_LIMIT)
res_he = find_best_saving_under_limit(dataset_to_test, wrapper_he, alphas, "Histogram Equalization", DIST_LIMIT)
res_gm = find_best_saving_under_limit(dataset_to_test, wrapper_gamma, gammas, "Gamma Correction", DIST_LIMIT)

# 5. Visualize final bar comparison chart
strategies = ['Hungry Blue', 'Brightness Scaling', 'Histogram Eq', 'Gamma Correction']
best_savings = [res_hb[0], res_bs[0], res_he[0], res_gm[0]]
best_params = [res_hb[1], res_bs[1], res_he[1], res_gm[1]]
actual_dists = [res_hb[2], res_bs[2], res_he[2], res_gm[2]]

plt.figure(figsize=(10, 6))
bars = plt.bar(strategies, best_savings, color=['#3498db', '#2ecc71', '#e74c3c', '#9b59b6'], alpha=0.85, edgecolor='black')

plt.title(f'Maximum Power Saving under Strict {DIST_LIMIT}% Distortion Limit', fontsize=16, fontweight='bold')
plt.ylabel('Average Power Saving (%)', fontsize=12)

# Reserve top space based on highest saving
max_y = max(best_savings)
plt.ylim(0, max_y + 10 if max_y > 0 else 10) 
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Add detailed data labels on bars
for bar, param, dist in zip(bars, best_params, actual_dists):
    height = bar.get_height()
    
    # Show saving percentage at top
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height:.2f}%', 
             ha='center', va='bottom', fontsize=12, fontweight='bold', color='black')
             
    # Show optimal parameter and actual distortion in the middle of bar
    if height > 0:
        info_text = f"Param: {param}\nDist: {dist:.2f}%"
        plt.text(bar.get_x() + bar.get_width()/2., height / 2,
                 info_text, 
                 ha='center', va='center', fontsize=11, color='white', fontweight='bold',
                 bbox=dict(facecolor='black', alpha=0.4, edgecolor='none', boxstyle='round,pad=0.3'))
    else:
        # If saving is 0 (e.g. HE cannot achieve it), label at bottom
        plt.text(bar.get_x() + bar.get_width()/2., 1, "No Saving", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import exposure, color

# ==========================================
# 1. Define wrapper functions for 4 strategies
# ==========================================

def wrapper_hungry_blue(img, k):
    img_mod = img.astype(np.int16)
    img_mod[:, :, 2] = img_mod[:, :, 2] - k
    return np.clip(img_mod, 0, 255).astype(np.uint8)

def wrapper_brightness(img, c):
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] *= c
    return (color.hsv2rgb(hsv) * 255).astype(np.uint8)

def wrapper_he(img, alpha):
    hsv = color.rgb2hsv(img)
    hsv_eq = hsv.copy()
    hsv_eq[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img_eq = (color.hsv2rgb(hsv_eq) * 255).astype(np.uint8)
    img_mixed = (img.astype(float) * (1 - alpha) + img_eq.astype(float) * alpha)
    return np.clip(img_mixed, 0, 255).astype(np.uint8)

def wrapper_gamma(img, gamma):
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.adjust_gamma(hsv[:, :, 2], gamma=gamma)
    return (color.hsv2rgb(hsv) * 255).astype(np.uint8)

# ==========================================
# 2. Configure parameters (optimized Hungry Blue range)
# ==========================================
strategies_config = {
    "Hungry Blue": {
        "func": wrapper_hungry_blue,
        "params": range(0, 101, 20), # k: 0, 20, 40, 60, 80, 100 (narrower range, more comparable)
        "style": {"color": "tab:blue", "marker": "o", "linestyle": "-"}
    },
    "Brightness Scaling": {
        "func": wrapper_brightness,
        "params": [1.0, 0.9, 0.8, 0.7, 0.6, 0.5], 
        "style": {"color": "tab:green", "marker": "s", "linestyle": "--"}
    },
    "Gamma Correction": {
        "func": wrapper_gamma,
        "params": [1.0, 1.2, 1.4, 1.6, 1.8, 2.0], 
        "style": {"color": "tab:purple", "marker": "D", "linestyle": "-"}
    },
    "Histogram Eq (Mix)": {
        "func": wrapper_he,
        "params": [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 
        "style": {"color": "tab:red", "marker": "^", "linestyle": ":"}
    }
}

# ==========================================
# 3. Compute data
# ==========================================
results_all = {}
print("Computing comparison data for 4 core strategies (based on images list)...")

for name, config in strategies_config.items():
    print(f"Running {name}...")
    avg_dists = []
    avg_savings = []
    
    for p in config["params"]:
        dists = []
        savings = []
        for img in images: # ensure 'images' list is loaded
            p_orig = estimate_power_part1(img)
            img_new = config["func"](img, p)
            p_new = estimate_power_part1(img_new)
            
            saving = (p_orig - p_new) / p_orig * 100 if p_orig > 0 else 0
            dist = compute_distortion(img, img_new)
            
            savings.append(saving)
            dists.append(dist)
            
        avg_dists.append(np.mean(dists))
        avg_savings.append(np.mean(savings))
        
    results_all[name] = {"x": avg_dists, "y": avg_savings}

# ==========================================
# 4. Plot (with boundary check)
# ==========================================
plt.figure(figsize=(10, 7))

# Define visible range
X_LIMIT = 8.0
Y_LIMIT = 40.0

for name, data in results_all.items():
    style = strategies_config[name]["style"]
    
    # Draw lines and points
    plt.plot(data["x"], data["y"], 
             label=name,
             color=style["color"],
             marker=style["marker"],
             linestyle=style["linestyle"],
             linewidth=2, markersize=7, alpha=0.8)
    
    # Annotate key parameter points
    params = strategies_config[name]["params"]
    # Annotate points at index 1, 2, 3 (adjust based on param count)
    indices_to_label = list(range(len(params))) 
    
    if len(params) > max(indices_to_label):
        for idx in indices_to_label:
            x_pos = data["x"][idx]
            y_pos = data["y"][idx]
            p_val = params[idx]
            
            # [Key fix] only draw label if point is within X axis range
            if x_pos <= X_LIMIT and y_pos <= Y_LIMIT:
                p_str = f"{p_val:.1f}" if isinstance(p_val, float) else f"{p_val}"
                
                # Fine-tune position to avoid overlapping with point
                plt.text(x_pos, y_pos + 0.8, 
                         p_str, 
                         fontsize=9, color=style["color"], fontweight='bold', ha='center')

# Add distortion constraint reference line
plt.axvline(x=2.0, color='gray', linestyle='-', alpha=0.3, label='Limit 2%')
plt.axvline(x=3.0, color='gray', linestyle='--', alpha=0.3, label='Limit 3%')

plt.title('Pareto Curves: Comparison of Core Strategies', fontsize=14)
plt.xlabel('Average Distortion (%)', fontsize=12)
plt.ylabel('Average Power Saving (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(fontsize=11, loc='upper left')

# Set axis range
plt.xlim(0, X_LIMIT)  
plt.ylim(0, Y_LIMIT)

plt.tight_layout()
plt.show()

In [ ]:
# Example: modify loop logic to support grouped analysis
results_natural = []
results_screens = []

for k in k_values:
    nat_savings, nat_dists = [], []
    scr_savings, scr_dists = [], []
    
    # Natural image group
    for img in images:
        p_orig = estimate_power_part1(img)
        img_new = apply_hungry_blue(img, k)
        p_new = estimate_power_part1(img_new)
        nat_savings.append((p_orig - p_new) / p_orig * 100)
        nat_dists.append(compute_distortion(img, img_new))
        
    # Screenshot group
    for img in screen_images:
        p_orig = estimate_power_part1(img)
        img_new = apply_hungry_blue(img, k)
        p_new = estimate_power_part1(img_new)
        scr_savings.append((p_orig - p_new) / p_orig * 100)
        scr_dists.append(compute_distortion(img, img_new))
        
    results_natural.append((np.mean(nat_dists), np.mean(nat_savings)))
    results_screens.append((np.mean(scr_dists), np.mean(scr_savings)))

In [ ]:
plt.figure(figsize=(10, 6))
# Plot natural image group
x_nat = [r[0] for r in results_natural]
y_nat = [r[1] for r in results_natural]
plt.plot(x_nat, y_nat, 'b-o', label='Natural Images (Average)')

# Plot screenshot group
x_scr = [r[0] for r in results_screens]
y_scr = [r[1] for r in results_screens]
plt.plot(x_scr, y_scr, 'r--s', label='Screenshots (Average)')

plt.title('Pareto Frontier: Natural vs. Screenshots')
plt.xlabel('Distortion (%)')
plt.ylabel('Power Saving (%)')
plt.legend()
plt.show()

In [ ]:
import pandas as pd

img49 = images[49]
p_orig = compute_power(img49)

# Check actual pixel mean and power share for each channel
r49, g49, b49 = img49[:,:,0].astype(np.float64), img49[:,:,1].astype(np.float64), img49[:,:,2].astype(np.float64)
gamma_pow = 0.7755
w_r, w_g, w_b = 2.13636845e-7, 1.77746705e-7, 2.14348309e-7

p_r = w_r * np.sum(np.power(r49, gamma_pow))
p_g = w_g * np.sum(np.power(g49, gamma_pow))
p_b = w_b * np.sum(np.power(b49, gamma_pow))

print(f"=== Image 49 Channel Analysis ===")
print(f"R: mean={r49.mean():.1f}, power share={p_r/p_orig*100:.1f}%")
print(f"G: mean={g49.mean():.1f}, power share={p_g/p_orig*100:.1f}%")
print(f"B: mean={b49.mean():.1f}, power share={p_b/p_orig*100:.1f}%")
print()

# Compare 3 strategies parameter by parameter
rows = []
for k in [0, 20, 40, 60, 80, 100]:
    img_new = wrapper_hungry_blue(img49, k)
    p_new = compute_power(img_new)
    rows.append({"Strategy": "Hungry Blue", "Param": f"k={k}",
                 "Saving(%)": round((p_orig-p_new)/p_orig*100, 3),
                 "Distortion(%)": round(compute_distortion(img49, img_new), 3)})

for c in [1.0, 0.9, 0.8, 0.7, 0.6, 0.5]:
    img_new = wrapper_brightness(img49, c)
    p_new = compute_power(img_new)
    rows.append({"Strategy": "Brightness", "Param": f"c={c}",
                 "Saving(%)": round((p_orig-p_new)/p_orig*100, 3),
                 "Distortion(%)": round(compute_distortion(img49, img_new), 3)})

for g in [1.0, 1.2, 1.4, 1.6, 1.8, 2.0]:
    img_new = wrapper_gamma(img49, g)
    p_new = compute_power(img_new)
    rows.append({"Strategy": "Gamma", "Param": f"γ={g}",
                 "Saving(%)": round((p_orig-p_new)/p_orig*100, 3),
                 "Distortion(%)": round(compute_distortion(img49, img_new), 3)})

df = pd.DataFrame(rows)
print(df.to_string(index=False))
